In [1]:
import configparser
import datetime as dt
import sys
import collections
from pathlib import Path
from typing import Union, Optional, List, Tuple

import codebook.EDA as EDA
import codebook.clean as clean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('raph-base')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

np.random.seed(666)

In [3]:
Path.cwd().parent / 'src'

WindowsPath('C:/Users/r2d4/OneDrive/code/projects/20-02_disco/src')

In [4]:
sys.path.append(str(Path.cwd().parent))

# from src.db_declaration import (
#     Base, Artist, CreditTrx, Genre, Label, Record, RecordFormat, CreditTrx, RecordLabelLink, ArtistLabelLink
# )
from src import db_functions
from src import utils

<class 'sqlalchemy.ext.declarative.api.DeclarativeMeta'>


In [5]:
print(sys.executable)
print(sys.version)
print(f'sqlalchemy {sqlalchemy.__version__}')

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
sqlalchemy 1.3.17


## Connect To DB

In [6]:
CONFIG_PATH = Path.cwd().parent / "config.cfg"

path_to_db = utils.read_config_return_str(CONFIG_PATH, "SQLITE")
engine = utils.create_engine(path_to_db)

C:\Users\r2d4\OneDrive\code\projects\20-02_disco\db_aka_discobase\DiscoBase.db


## Query DB

In [7]:
pd.read_sql("records", engine).tail()

,record_id,artist_id,title,genre_id,label,year,format_id,vinyl_color,lim_edition,number,remarks,purchase_date,price,digitized,rating,active,created_at,updated_at
288,289,186,Splinters From An Ever-Changing Face,5,None,2020,None,purple translucent with white & yellow moon phase,500,None,2nd Press,2020-10-01,25,1,nan,1,2021-01-06 20:51:26,NaT
289,290,186,From The Unforgiving Arms Of God,5,None,2017,None,clear with gold (metallic) marble,250,None,6th Press (2020),2020-10-01,25,1,nan,1,2021-01-06 20:51:26,NaT
290,291,94,A New Realm Of Misery,5,None,2019,None,pink green,100,None,None,2020-10-01,50,0,nan,1,2021-01-06 20:51:26,NaT
291,292,187,Carved In Flesh,5,None,2018,None,None,None,None,None,2020-11-01,20,1,nan,1,2021-01-06 20:51:26,NaT
292,293,188,We Move As One,5,None,2020,None,clear,None,None,None,2020-12-01,20,1,nan,1,2021-01-06 20:51:26,NaT


In [8]:
pd.read_sql("credit_trx", engine)

,credit_trx_id,credit_trx_date,credit_trx_type,credit_value,credit_saldo,record_id,created_at,updated_at
0,1,1992-01-01,Purchase,0.00,0.00,1,2021-01-06 20:51:15,NaT
1,2,1992-01-01,Purchase,0.00,0.00,2,2021-01-06 20:51:15,NaT
2,3,1992-01-01,Purchase,0.00,0.00,3,2021-01-06 20:51:15,NaT
3,4,1992-01-01,Purchase,0.00,0.00,4,2021-01-06 20:51:15,NaT
4,5,1992-01-01,Purchase,0.00,0.00,5,2021-01-06 20:51:15,NaT
...,...,...,...,...,...,...,...,...
288,289,2020-10-01,Purchase,0.00,0.00,289,2021-01-06 20:51:26,NaT
289,290,2020-10-01,Purchase,0.00,0.00,290,2021-01-06 20:51:26,NaT
290,291,2020-10-01,Purchase,0.00,0.00,291,2021-01-06 20:51:26,NaT
291,292,2020-11-01,Purchase,0.00,0.00,292,2021-01-06 20:51:26,NaT


In [9]:
pd.read_sql("record_label_link", engine)
pd.read_sql("artist_label_link", engine)
pd.read_sql("artist_genre_link", engine)
pd.read_sql("genre_label_link", engine)

,record_id,label_id
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
288,289,38
289,290,38
290,291,39
291,292,40


,artist_id,label_id
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
194,185,27
195,186,38
196,94,39
197,187,40


,artist_id,genre_id
0,1,1
1,2,1
2,3,2
3,4,1
4,5,2
...,...,...
185,184,1
186,185,1
187,186,5
188,187,5


,genre_id,label_id
0,1,1
1,2,1
2,3,1
3,4,1
4,1,2
5,5,1
6,1,3
7,1,4
8,1,5
9,1,6
